In [90]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats import endpoints
import pandas as pd

pg = players.find_players_by_full_name('paul george')[0]
pg_id = pg['id']

# get top 8 pg teammates with most mins played

clippers = teams.find_team_by_abbreviation('lac')
clippers_id = clippers['id']
clippers_roster = endpoints.teamplayerdashboard.TeamPlayerDashboard(team_id=clippers_id, season='2023-24').get_data_frames()[0]
clippers_roster_data = endpoints.commonteamroster.CommonTeamRoster(team_id=clippers_id, season = '2023-24').get_data_frames()[0]


player_minutes = []

# get the minutes for each player

for index, player in clippers_roster_data.iterrows():
    if player['PLAYER_ID'] != pg_id:
        player_id = player['PLAYER_ID']
        player_name = player['PLAYER']
        game_log = endpoints.playergamelog.PlayerGameLog(player_id=player_id, season='2023-24').get_data_frames()[0]
        total_minutes = game_log['MIN'].sum()
        player_minutes.append({'name': player_name, 'minutes': total_minutes, 'id': player_id})


# sort top 8

minutes_df = pd.DataFrame(player_minutes)
top_8 = minutes_df.sort_values('minutes', ascending=False).head(8)

top_8_id = []
for index, player in top_8.iterrows():
    top_8_id.append((player['id']))


# get the two pt and three pt for each player

top_8_shooting_splits = {}

for top_8_player_id in top_8_id:
    top_8_player_stats = endpoints.playerdashboardbygeneralsplits.PlayerDashboardByGeneralSplits(player_id=top_8_player_id, season='2023-24').get_data_frames()[0]
    three_pt_percent = top_8_player_stats['FG3_PCT'][0]
    two_pt_percent = (top_8_player_stats['FGM'][0] - top_8_player_stats['FG3M'][0]) / (top_8_player_stats['FGA'][0] - top_8_player_stats['FG3A'][0])
    two_three_array = [two_pt_percent, three_pt_percent]
    top_8_shooting_splits[top_8_player_id] = two_three_array

    
print(top_8_shooting_splits)

    













{201935: [0.49700598802395207, 0.381], 202695: [0.5690072639225182, 0.417], 1626181: [0.5364583333333334, 0.435], 1629611: [0.6332179930795848, 0.348], 1627826: [0.649325626204239, 0.0], 201566: [0.5088408644400786, 0.273], 1629599: [0.5714285714285714, 0.38], 1628464: [0.5855855855855856, 0.366]}
